In [1]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

In [15]:
dframe = pd.read_csv('../data/imdb_all_v2.csv')
df = dframe[(10 <= dframe['star_index']) & (dframe['star_index'] <= 20)]
genre_list = []
for i in range(len(df)):
    try:
        # genre_list.append(df.iloc[i]['genre'].split(','))
        genre_list.append(df.iloc[i]['certificate'].split(','))
    except:
        genre_list.append(None)
df['genre_list'] = genre_list
df = df.explode('genre_list')
df['genre_list'] = df['genre_list'].apply(lambda x: x.strip() if x != None else x)
genre_group = df.groupby('genre_list')

genres = []
number = []
years = []
for genre in genre_group:
    # for year_group in genre[1].groupby('year'):
    for i in range(1920, 2022):
        years.append(i)
        genres.append(genre[0])
        number.append(len(genre[1][genre[1]['year'] == i]))
output = pd.DataFrame(data = [genres, number, years]).T
output.columns = ['genre', 'count', 'year']

<ipython-input-15-f25466437038>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [16]:
trace_dict = {}
all_genre = output['genre'].unique()

for genre in all_genre:
    trace_dict[genre] = go.Scatter(
                            x= output['year'], y = output[output['genre'] == genre]['count'],
                            name = genre,
                            mode = 'none', 
                            stackgroup = 'one', line_shape = 'spline')

In [17]:
fig = go.Figure()
for genre in trace_dict.keys():
    fig.add_trace(trace_dict[genre])

fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)
fig.show()